# Bedrock Client Prompt Cache Testing Notebook

This notebook tests the `invoke_model` method in the Bedrock client (`genaiic-idp-accelerator/lib/idp_common_pkg/idp_common/bedrock/client.py`). It includes tests for prompt caching.

Prompt caching is a powerful feature in Amazon Bedrock that significantly reduces response latency for workloads with repetitive contexts. This notebook demonstrates how to implement prompt caching for IDP prompts.

Prompt caching allows you to store static portions of your prompt, enabling models to:
- Reuse cached context instead of reprocessing inputs
- Reduce response Time-To-First-Token (TTFT) for subsequent queries

Benefits of Prompt Caching
- Faster Response Times: Avoid reprocessing the same context repeatedly
- Cost Efficiency: Potentially lower token usage by avoiding redundant processing

NOTE: To effectively use Prompt Caching, there is a [minimum number of tokens](https://docs.aws.amazon.com/bedrock/latest/userguide/prompt-caching.html#prompt-caching-models) for the cache. 

## 1. Setup and Imports

First, we'll import the necessary libraries and set up logging to see debug output.

In [ ]:
# Let's make sure that modules are autoreloaded
%load_ext autoreload
%autoreload 2

ROOTDIR="../.."
# First uninstall existing package (to ensure we get the latest version)
%pip uninstall -y idp_common

# Install the IDP common package with all components in development mode
%pip install -q -e "{ROOTDIR}/lib/idp_common_pkg[dev, all]"

# Note: We can also install specific components like:
# %pip install -q -e "{ROOTDIR}/lib/idp_common_pkg[ocr,classification,extraction,evaluation]"

# Check installed version
%pip show idp_common | grep -E "Version|Location"

# Optionally use a .env file for environment variables
try:
    from dotenv import load_dotenv
    load_dotenv()  
except ImportError:
    pass  

Found existing installation: idp_common 0.3.4
Uninstalling idp_common-0.3.4:
  Successfully uninstalled idp_common-0.3.4
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Version: 0.3.4
Location: /home/ec2-user/.local/lib/python3.12/site-packages
Note: you may need to restart the kernel to use updated packages.


In [2]:
import logging
from typing import Dict, Any

# Import the Bedrock client
from idp_common.bedrock.client import BedrockClient, invoke_model

# Configure logging 
logging.basicConfig(level=logging.WARNING)  # Set root logger to WARNING (less verbose)
logging.getLogger('idp_common.bedrock.client').setLevel(logging.DEBUG)  # show prompts

### Helper Functions

Let's create some helper functions to make testing easier.

In [3]:
def print_response_summary(response: Dict[str, Any]) -> None:
    """Print a summary of the Bedrock response."""
    if not response:
        print("Empty response")
        return
        
    # Extract the actual response from the wrapper
    bedrock_response = response.get("response", {})
       
    # Print usage information
    if "usage" in bedrock_response:
        usage = bedrock_response["usage"]
        print(f"Usage: {usage}")


## 1. Test the CachePoint Preprocessing Function

First, let's test the preprocessing function directly to see how it handles various inputs.

In [18]:
# Create a client instance
client = BedrockClient()

# Test 1: Basic cachepoint in middle of text
content1 = [{"text": "This is the first part.<<CACHEPOINT>>This is the second part."}]
print("\nTest 1 - Basic cachepoint in middle:")
processed1 = client._preprocess_content_for_cachepoint(content1)
print(processed1)


DEBUG:idp_common.bedrock.client:Found <<CACHEPOINT>> tags in text content: This is the first part.<<CACHEPOINT>>This is the s...
DEBUG:idp_common.bedrock.client:Split text into 2 parts at cachepoint tags
DEBUG:idp_common.bedrock.client:Text part 1: 5 words
DEBUG:idp_common.bedrock.client:Inserting cachePoint #1 after text part 1
DEBUG:idp_common.bedrock.client:Text part 2: 5 words
INFO:idp_common.bedrock.client:Processed content with 1 cachepoint insertions



Test 1 - Basic cachepoint in middle:
[{'text': 'This is the first part.'}, {'cachePoint': {'type': 'default'}}, {'text': 'This is the second part.'}]


## 2. Test Caching in an typical classification inference

Now we'll test the `invoke_model` method with a Claude model using a prompt with a <<CACHEPOINT>> tag.

In [19]:

# Typical cl;assification system prompt
system_prompt = 'You are a document classification expert who can analyze and classify multiple documents and their page boundaries within a document package from various domains. Your task is to determine the document type based on its content and structure, using the provided document type definitions. Your output must be valid JSON according to the requested format.'

# Typical classification task prompt arranged to have static content first, separated from dynamic content by <<CACHEPOINT>> tag
task_prompt = """
The <document-types> XML tags contain a markdown table of known doc types for detection.
<document-types>
| type | description |\n| --- | --- |\n| letter | A formal written message that is typically sent from one person to another |\n| form | A document with blank spaces for filling in information |\n| email | An electronic message sent from one person to another over a computer network |\n| specification | A detailed description of technical requirements or characteristics |\n| memo | A brief written message used for internal communication within an organization |\n| invoice | A commercial document issued by a seller to a buyer relating to a sale |\n| resume | A document summarizing a person\'s background, skills, and qualifications |\n| scientific_publication | A formally published document presenting scientific research findings |\n| advertisement | A public notice promoting a product, service, or event |\n| questionnaire | A set of written questions designed to collect information from respondents |\n| generic | A general document type that doesn\'t fit into other specific categories |
</document-types>
<guidance>
Guidance for terminology found in the instructions.\n    * ordinal_start_page: The one based beginning page of a document segment within the document package.\n    * ordinal_end_page: The one based ending page of a document segment within the document package.\n    * document_type: The document type code detected for a document segment.\n    * Distinct documents of the same type may be adjacent to each other in the packet. Be sure to separate them into different document segments and don\'t combine them.
</guidance>
<document-splitting-guidance>
When deciding whether pages belong to the same document segment:\n- Content continuity: Pages with continuing paragraphs, numbered sections, or ongoing narratives likely belong to the same document.\n- Visual/formatting consistency: Similar layouts, headers, footers, and styling suggest pages belong together.\n- Logical completion: A document typically has a beginning, middle, and end structure.\n- Document boundaries: Look for clear indicators of a new document such as new title pages, cover sheets, or significantly different subject matter.\n- Content similarity: Pages discussing the same topic or subject likely belong to the same document.\n\nPages should be grouped together when they represent a coherent, continuous document, even if they span multiple pages. Split documents only when there is clear evidence that a new, distinct document begins.
</document-splitting-guidance>
CRITICAL: You must ONLY use document types explicitly listed in the <document-types> section. Do not create, invent, or use any document type not found in this list. If a document doesn\'t clearly match any listed type, assign it to the most similar listed type or "other" if that option is provided.\n\nFollow these steps when classifying documents within the document package:\n1. Examine the document package as a whole, and identify page ranges that are likely to belong to one of the <document-types>.\n2. Match each page range with an identified document type from the provided list ONLY.\n3. Identify documents of the same type, that are not the same document but are adjacent to each other in the packet.\n4. Separate unique documents of the same type adjacent to each other in the packet into distinct document segments. Important: Do not combine distinct documents of the same type into a single document segment.\n5. For each identified document type, note the ordinal_start_page and ordinal_end_page.\n6. Compile the classified documents into a list with their respective ordinal_start_page and ordinal_end_page.\n7. Before finalizing, verify that each document type in your response exactly matches one from the <document-types> list.\n
Return your response as valid JSON according to this format:
```json\n{\n    "segments": [\n                      {\n                        "ordinal_start_page": 1,\n                        "ordinal_end_page": 2,\n                        "type": "the first type of document detected"\n                      },\n                      {\n                        "ordinal_start_page": 3,\n                        "ordinal_end_page": 4,\n                        "type": "the second type of document detected"\n                      }\n                    ]\n}
```
The <document-text> XML tags contains the text separated into pages from the document package. Each page will begin with a <page-number> XML tag indicating the one based page ordinal of the page text to follow.

<<CACHEPOINT>>

<document-text>
<page-number>1</page-number>\n\n\nWESTERN DARK FIRED TOBACCO GROWERS\' ASSOCIATION \n\n206 Maple Street P.O. Box 1056 Murray Kentucky 42071-1056 \n\n(502) 753-3341 FAX (502) 753-0069/3342 \n\nOctober 31, 1995 \n\nThe Honorable Wendell H. Ford United States Senate Washington, D. C. 20510 \n\nDear Senator Ford: \n\nOn behalf of the Western Dark Fired Tobacco Growers\' Association and the 9,000 tobacco producers it represents, I an obligated to convey our strong opposition to the "Commitment to our Children\' petition being circulated by several Members of Congress. \n\nIn the tobacco industry, no one wants young people to consume tobacco products and age restriction laws are on the books in every state in the nation. We must take action to better enforce these laws, not create more bureaucracy. \n\ngovernment bureaucracy, thus destroying our family farms and There are those in our society who want to add inefficient hampering an adult\'s First Amendment right to the freedom of choice in using a legal product. \n\nYou may have been approached by those who say they are supporting Administration (FDA) regulation of tobacco. However, if FDA is given the authority to regulate tobacco because it is a nicotine the cause of youth smoking prevention by pushing the Food and Drug delivery device\', farmers will be forced to deal with yet another monitored by the United States Department of Agriculture the government agency. Already our producers deal with and are Administration and many others. We know first hand what a Environmental Protection Agency, the Occupational Safety and Health We certainly need your support and involvement to prevent FDA from nightmare federal government regulations can create for farmers. joining the ranks of federal tobacco regulators. \n\nChildren" petition, looking at it for what it is: more anti-tobacco I urge you to consider the consequences of the "Commitment to Our indoctrination, rather than a solution to a problem which everyone starting with parents, should address in a responsible manner. \n\nSincerely, Will E tlack Will E. Clark General Manager \n\nTNJB 0008497\n\n\n<page-number>2</page-number>\n# LAB SERVICES CONSISTENCY REPORT \n\nDATE: 2/28/93\n TECHNICIAN: CC\n SHIFT: A\nTrial 8\n LINE: 2\n AREA: 52\nPRODUCT UNIT CODE:\n0728\n SAMPLE ID:\nstuff box 2\n REASON FOR REQUEST\ntest\n\nREQUESTED DELIVERY TIME:\n\nTIME SAMPLE RECEIVED:\n-\n TIME ANALYSIS COMPLETED:\n\nDATA COMMUNICATED TO\nGone\nAT 105\n\nPerson\nTime\n\nDRYING TIME\n\n\n\n\nA\tB\tC\tD\tE\tIN:\tOUT:\nSAMPLE &\tCONTAINER\tSAMPLE\tDILUTION\tDILUTED\nCONTAINER\tWEIGHT IN\tWEIGHT IN\tFACTOR\tSAMPLE\nWEIGHT IN\tGRAMS\tGRAMS\tWEIGHT IN\nGRAMS\t(A-B)\tGRAMS\nF\tG\tH\tI\n1159.3\t6\t6955. 8\tWEIGHT IN\tPAPER\tFILTER\tWEIGHT IN\tORAMS\tSAMPLE\tWEIGHT IN\tPAPER\tSAMPLE &\tH-P\t% CONSISTENCY\to\tx D 1 00\nORAMS\tGRAMS\nWET\tDRY\nAVERAGE 3.68\tCONSISTENCY\tSAMPLE #1 10\tSAMPLE #7 //\t2645\t2853\t84 568\t79.235\t3.366\t3123\t3.62\t3.64\n15\nSAMPLE $ 12\t2847\t89.776\t3.411\t3.77\nRANGE:\nMETER READING:\nCOMMENTS:\n\n\n\nACONSIST wis\n\n\n2030053328\n\n\n<page-number>3</page-number>\n# Ashley Bratich \n\nFrom: Kelahan, Ben To: TI New York: \'TI Minnesota Co: Ashley Bratich (MSMAIL) Subject: FW: Morning Team Notes 4/20 Date: Saturday, April 18. 1998 2:09PM \n\nOriginal Message From: Byron Nelson (SMTP:bnelson@wka.com] Sent: Friday, April 17. 1998 5:25 PM To: Judy Albert: Carolyn: Jackie Cohen (AWMA): Frank: Goody; Henry: Hollant: Chris Holt; Hurst: Jim; Joe: John; Benjamin Kelahan: Cheryl Klein: Walt Klein: Lbeckwith; Rob Meyne: Mkatz; Morrow; Powers; Randy; Roger; Ron: Shorep; Steve Strawsburg: Suggsm; Matthew Tilley: whitey Co: Bob Fackler: Bob Stone Subject: Morning Team Notes 4/20 \n\nFalmouth MA - On 4/15, town meeting representatives defeated by a 84-77 vote a warrant article calling for a 100 percent ban on smoking in restaurants On 4/16, a motion to reconsider the vote was soundly rejected 104-49. The restaurant owner\'s moderate alternative was not considered because the town counsel found the article to be unconstitutional \n\nWaseca County MN - On 4/7 the county commissioners once again tabled consideration of a new tobacco retailing ordinance Waseca is the 11th Minnesota community to put the issue on hold. \n\nWadena County, MN - In mid-March the county commissioners tabled consideration of a new tobacco retailing ordinance until 4/23 At that time. they will take up a model ordinance that mirrors the state law. Bob Fackler requests calls to retailers to alert them to attend \n\nPage 1\n\n\nTI1716-0284\n\n\n<page-number>4</page-number>\nLE CHOIHGPRT Mutation ussey - Alyoral 40LF Book No. 354 \n\nin Page No 85, NB. 343 \n\nObjectives To measure the ability of a test substance to induse mutation\n at the hypoxanthine gunnine prosphonloryl transferes (hgpat) lear\n in Chinese Hamster very (CHO) cells on the basis that\n mutants by viture of the loss of the HGPRT activity are unabe\n to convert purine analogs, such as a thingsamine (6-ta) to\n toxic metabilities and hence escape their lethal effects which\n is however, excountered ly the wild type cells.\n Naturals and Methods Refer to Standard Operating Procedure PH314.\n Sponsor: American Cyanamid Company\n Test Citiete Algoral 40 LF\n Descuption clean liquid\n Date Prelimining Cytotorist Instrated 6/3/82\n Date CHOIHGAET forward bene Mutation Assay Instituted 8/26/82\n CHO- KI-BH4 Jost 4 7182 received from Oak Ridge national Laborations 7/1/2\n Routine subculture were done every Friday (a.m.) and Monday (p.m.),\n where 1x105 cells were subcultined into each of 3-75cm flasks\n containing IS ml of media FIZFOSIO. CHO KI-BHY fat# 7182 animption\n treated 7/23/82. Routine subcolture regine camid out.\n 8/23/82 CHO-KI-BHY cells (for#7182) subcultured into 10-T75cm2\n flasks (3x10 cells/flook) in IS mlof media FIRESID\n\nEggadek 8/23/82\n\n8/25/82. CHO-KI.BH4 cells subcultured into 36 - T25 cm2\n flasks (5x105 cells (flask) in 5ml of media F12 FCMS, in\n puparation for treatment (2/21/88) FetalBoundem Lvd KC-32 1005\n\nTo Page No. 6\n\nitnessed & Understood by me,\nDate\nInvented by\nDate\n knog neech\n8/25/82\nRecorded\nEdmind by D. Good\n8/25/82\n\n\n<page-number>5</page-number>\n\n\nPeake Printers, Inc. 2500 Schuster Drive Cheverly, Maryland 20781 (301) 341-4600 WASH. 1-800-521-PEAK (301) 792-2704 BALT. (301) 341-1162 FAX \n\n# INVOICE \n\nBILL TO \n\n20050 THE TOBACCO INSTITUTE ATTN: ANNE CANNELL 1875 I STREET, N.W. WASHINGTON DC 20006 \n\nTHE TOBACCO INSTITUTE ATTN: ANNE CANNELL 1875 I STREET N.W. WASHINGTON DC 20006 \n\n*** INVOICE *** \n\nInvoice No: 86239 Invoice Date: 11/12/92 Ship Date: 10/13/92 P.O. Number: Salesman MICHAEL J MCKILLIPS Job Number: 86239 Ship Via: Terms: NET 30 DAYS \n\nA Service Charge of 2% per month (24% per year) will be charged if payment not received by end of first month after invoice date \n\n\n\nDESCRIPTION\tQUANTITY\tUL.\tM\tUNIT PRICE\tAMOUNT\nTWO SIDED DECAL "IT\'S THE\t5000\t5145.000\t5145.00\nLAW--UNDER 18" PRINTS 2/2,\n5 1/2 x 7 1/2"\nSUB TOTAL\t5145.00\nTAX\t308.70\nTOTAL INVOICE\t5453.70\nInvoice\tAMT DUE\t5453.70\n83829\tLESS DEPOSIT\t(11000.00)\nCREDIT BALANCE\t$5546.30\nok- ADCOH\nfrom 1501 5201\nsee attached original\ndeposit / invoice\nCONFIDENTIAL:\nMINNESOTA TOBACCO LITIGATION\n\n\n\nTIMN 0163588 \n\nCUSTOMER COPY FEDID +52-0784214 DUNS #003244142 \n\n1\n\n\nForm 700 7.88 \n\n<page-number>6</page-number>\nNEWS RELEASE SIZE Dixon DISSENT FEDERAL TRADE COMMISSION Washington, D.C. 20580 Rein 8- 4-67 OFFICE OF INFORMATION 393-6800 Ext. 197 For RELEASE: A.M., Tuesday, August 1, 1967 \n\n# FTC TO BEGIN CIGARETTE TESTING \n\nThe Federal Trade Commission, having been advised by the staff that the cigarette testing laboratory has satisfactorily completed its trial tests, has now issued directions to commence the first formal test, under the follow- ing conditions: \n\n1. Smoke cigarettes to a 23 - butt length, or to the length of the filter and overwrap plus 3 mm. if in excess of 23 -. \n2. Base results on a test of 100 cigarettes per brand, or type, \n3. Cigarettes to be tested will be selected on a random basis, as opposed to "weight selection," \n4. Determine particulate matter on a "dry" basis employing the gas chromatography method published by C. H. Sloan and B. J. Sublett in Tobacco Science 9, page 70, 1965, as modified by F. J. Schultz\' and A. W. Spears\' report published in Tobacco Vol. 162, No. 24, page 32, dated June 17, 1966, to determine the moisture content, \n5. Determine and report the "tar" content after subtracting moisture and alkaloids (as nicotine) from particulate matter. \n6. Report tar content to the nearest whole milligram and nicotine content to the nearest 1/10 milligrams. \nThe Commission directed that the test cover approximately 50 of the major brands and types (many brands are sold as regular, and king size, or filter, etc.) and all brands for which any tar or nicotine statement appears on the label or in the advertising. With respect to the latter, one purpose of the test will be to determine the accuracy of such statement. Cigarettes for testing will be purchased on the open market in 50 localities throughout the United States. \nIn determining the foregoing procedures, the Commission relied substan- tially upon a record including written presentation by interested persons and oral testimony offered at a public hearing on November 30, 1966, which was held "to assist the Commission in determining what action, if any, should be taken in the public interest with respect to modifying or amplifying the Cambridge Filter Method. and the form in which test results should be expressed." At the hearing the Commission received numerous submissions re- flecting a variety of modifications of the Cambridge Filter Method that have been adopted by different groups engaged in testing cigarettes. No test can precisely duplicate conditions of actual human smoking and, within fairly wide limits, no one method can be said to be either "right" or "wrong." The Commission considers it most important that the test results be based on a reasonable standardized method and that they be capable of being presented to the public in a manner that is readily understandable. Although minor variations may not make one testing method "better" than another, the public \nyour to Mission Byown Apples a Wolfe, Lane A7h7 \n00 \n\nfrom Public Health ack\n\nBearing FTC July hise adv\n\n\n<page-number>7</page-number>\n# PLEASE TELL US WHAT YOU THINK \n\nHow satisfied were you in each of the following areas: \n\nNeither\n\nVery Somewhat Satisfied Nor Somewhat\nVery\n Satisfied Satisfied Dissatisfied Dissatisfied Dissatisfied\n\n1. PHONE CALL\n\nWas our representative\n courteous and polite?\n\nWas our representative\n knowledgeable?\n\nWas your question/request\n handled?\n\n2. Which ONE of the following statements BEST describes the way you feel about R. J.\n Reynolds\' response to your request for assistance?\n\nI was very satisfied\nRequested a Catalog\n I was somewhat satisfied\n\nI was neither satisfied nor dissatisfied\n\nI was somewhat dissatisfied\n\nI was very dissatisfied\n\n3. Based on the service you received, will you continue to purchase the brand of cigarettes\n you contacted us about?\n\nI Definitely\nI Probably\nI Might or\nI Probably\nI Definitely\n Would\nWould\nMight Not\nWould Not\nWould Not\n\n4. Based on the service you received, would you recommend this brand of cigarettes to an\n adult smoker (21 years of age or older) who currently smokes a competitive brand?\n\nI Definitely I Probably I Might or\nI Probably\nI Definitely\n Would\nWould\nMight Not\nWould Not\nWould Not\n\n\n52435 9399 \n\n<page-number>8</page-number>\n# BIOGRAPHICAL SKETCH \n\nGive the following information for the key personnel and consultants listed on page 2 Begin with the Principal Investigator/Program Director Photocopy this page for each person \n\n\n\nNAME\tPOSITION TITLE\tBIRTHDATE (Mo. Day, Yr.)\nMario Stevenson\tAssistant Professor\tMay 11, 1957\n\n\n\n\n\nEDUCATION (Begin with baccalaureate or other initial professional education such as nursing and include postdoctoral training)\nINSTITUTION AND LOCATION\nYEAR\nDEGREE\tCONFERRED\tFIELD OF STUDY\nGlasgow College of Technology\tB.Sc.\t1979\tBiochemistry\nGlasgow, Scotland\nUniversity of Strathclyde\tPh.D.\t1984\tBiochemistry\nGlasgow, Scotland\n\n\n\nRESEARCH AND PROFESSIONAL EXPERIENCE Concluding with present position list in chronological order, previous employment, experience and honors. Include present membership on any Federal Government public advisory committee List. in chronological order. the titles and com plete references to all publications during the past three years and to representative earlier publications pertinent to this application DO NO EXCEED TWO PAGES \n\n## EXPERIENCE \n\n8/80 5/84: Research Associate, Department of Pharmacology, University of Strathclyde, Glasgow, Scotland. 10/84 5/86: Research Fellow, Department of Pathology & Microbiology, University of Nebraska Medical Center Omaha, Nebranka 6/86 6/87: Instructor, Department of Pathology & Microbiology University of Nebracka Medical Center Omaha, Nebraska 7/87 - Present Assistant Professor, Department of Pathology & Microbiology, University of Nebraska Medical Center Omaha, Nebraska \n\n## HONORS \n\nBritish Pharmaceutical Association Travel Award. Glasgow College of Technology, Bachelor of Science with Honors. \n\n## PUBLICATIONS \n\n1. Stevenson, M., Baillie, A.J., and Richards, R.M.E. Enhanced activity of Streptomycin and Chloramphenicol against intracellular E. coli in the J774 macrophage cell line mediated by lipsome delivery. Antimicrob. Agents. Chenother. 24:742-749, 1985. \n2. Stevenson. M., Baillie, A.J., and Richards, R.M.E. An in-vitro model of intracellular bacterial infection using the murine macrophage cell line J774.2. J. Phar. Pharmacol. 36:90-94, 1984. \n3. Stevenson, M., Baillie, A.J., and Richards, R.M.E. Quantification of liposomal uptake in J774 macrophages -- a flow cytometric study. J. Pharm. Pharmacol. 38:120-126, 1985. \n4. Volsky, D.J., Wu. Y.T., Stevenson, M., Sinangil, F., Merino, F., Rodriguez, L., and Godoy, G. Antibodies to HTLV-III/LAV in Vene- zuelan patients with acute malaria infection (P. falciparum and P. vivax). New England J. Med., 314, #10:647-648, 1986. \n5. Shapiro. I., Stevenson, M., Sinangil, F., and Volsky, D.J. Transfection of lysphoblastoid cells: expression of co- transfected DNA and selection of transfected cell lines. Somatic cell & Mol. Genetics, 12:351-356, 1986. \n\n<page-number>9</page-number>\n# CURRICULUM VITAE \n\nSURNAME: Kalina BIRTHDATE January 21, 1938 \n\nFIRST NAME: Moshe \n\nEDUCATION BACKGROUND \n\n\n\nFrom-To\tInstitution\tArea of specialization\tDegree\n1958-1961\tHebrew University of Jerusalem\tAgriculture\tR So\n1961-1964\tHebrew University of Jerusalem\tBiochemistry\tM So\n1964-1967\tUniv. of London, King\'s College\tCytochemistry\tPh D.\n\n\n\nMajor research interest: The surfactant system: cell biological approach \n\n## EMPLOYMENT (start with present position) \n\n\n\nFrom-To\tInstitution\tResearch area\tTitle\n1978-present\tDept. of Histology, Tel Aviv niversity\tThe surfactant system\tAssoc Prof.\n1972-1978\tDept. of Histology & Cell Biology, -\tCytatoxic lymphocyte\tSen Lectur.\n1967-1978\tDept. of Histology&Cell Biology, 11\tHistochemistry\tLecturer\n1990-1991\tNational Jewish Hospital, Denver\tThe surfactant system\tVisit,Ass\n1984-1985\tPostgraduate Medical School London\tImmunory tochem-endocris Vist Ass.\nsystem\n1976-1977\tDept. of Anatomy, UCLA\tThe surfactant system\tVist Ass.\tProf\n1972-1973\tJohns Hopkins University\tE.M. cytochenistry\tVist,Ass.\tProf\nList grants and contracts on related or other subjects currently received by investigator from BSF and other sources.\nFrom-To\tTitle of project\tSources\tproject\t% Time/\tTotal grant\n\n\n\nPro \n\nProf \n\n50601911\n\n\n<page-number>10</page-number>\nFraserSmith THING MARKETING *STRATEGIC PLANNING . PROMOTION \n\n# MEMORANDUM \n\n\n\nTo:\tHoward Goldfrach\tDate:\tApril 3, 1987\nKathy Leiber\nFrom:\tMel Fallis\tRe:\tBlack Consumer\nMarket Promotion\nDevelopment\n\n\n\nAttached you will find the proposed project development timetable for the Situation Analysis and Campaign Development phases for Benson & Hedges and Virginia Slims. At the scheduled meetings on April 9th with Howard at 1:30PM and Kathy at 3PM, concentration will be placed on the subjects for review and evaluation identified for April. I am very interested in obtaining copies of as much information as available about the industry, category, consumer dynamics and promotion activities. \n\nIf there are questions prior to our meeting, please don\'t hesitate to call. \n\nMel \n\nCC: Ellen Merlo Terry Fraser Emmie LaBauve \n\n475 10th AVENUE . SUITE 800 . NEW YORK, N.Y. 10018 . (212) 564-8588
</document-text>
"""
content = [{'text': task_prompt}]


### Test bedrock inference with prompt cache point enabled - check usage output to verify cache behavior.

In [20]:
def invoke_with_caching(model_id, i):
    try:
        print(f"Testing invokation #{i}, with CACHEPOINT")
        response = client.invoke_model(
            model_id=model_id,
            system_prompt=system_prompt,
            content=content,
            temperature=0
        )
        
        print("\nResponse summary:")
        print_response_summary(response)
        
    except Exception as e:
        print(f"Error: {str(e)}")

In [21]:
# Test with claude
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

In [22]:
invoke_with_caching(model_id, 1)

Testing invokation #1, with CACHEPOINT


DEBUG:idp_common.bedrock.client:Found <<CACHEPOINT>> tags in text content: 
The <document-types> XML tags contain a markdown ...
DEBUG:idp_common.bedrock.client:Split text into 2 parts at cachepoint tags
DEBUG:idp_common.bedrock.client:Text part 1: 667 words
DEBUG:idp_common.bedrock.client:Inserting cachePoint #1 after text part 1
DEBUG:idp_common.bedrock.client:Text part 2: 2474 words
INFO:idp_common.bedrock.client:Processed content with 1 cachepoint insertions
INFO:idp_common.bedrock.client:Applied cachePoint processing for supported model: us.anthropic.claude-3-7-sonnet-20250219-v1:0
INFO:idp_common.bedrock.client:Bedrock request attempt 1/8:
DEBUG:idp_common.bedrock.client:  - model: us.anthropic.claude-3-7-sonnet-20250219-v1:0
DEBUG:idp_common.bedrock.client:  - inferenceConfig: {'temperature': 0, 'topP': 0.1}
DEBUG:idp_common.bedrock.client:  - system: [{'text': 'You are a document classification expert who can analyze and classify multiple documents and their page boundaries wit


Response summary:
Usage: {'inputTokens': 5100, 'outputTokens': 800, 'totalTokens': 6947, 'cacheReadInputTokens': 0, 'cacheWriteInputTokens': 1047}


In [23]:
invoke_with_caching(model_id, 2)

DEBUG:idp_common.bedrock.client:Found <<CACHEPOINT>> tags in text content: 
The <document-types> XML tags contain a markdown ...
DEBUG:idp_common.bedrock.client:Split text into 2 parts at cachepoint tags
DEBUG:idp_common.bedrock.client:Text part 1: 667 words
DEBUG:idp_common.bedrock.client:Inserting cachePoint #1 after text part 1
DEBUG:idp_common.bedrock.client:Text part 2: 2474 words
INFO:idp_common.bedrock.client:Processed content with 1 cachepoint insertions
INFO:idp_common.bedrock.client:Applied cachePoint processing for supported model: us.anthropic.claude-3-7-sonnet-20250219-v1:0
INFO:idp_common.bedrock.client:Bedrock request attempt 1/8:
DEBUG:idp_common.bedrock.client:  - model: us.anthropic.claude-3-7-sonnet-20250219-v1:0
DEBUG:idp_common.bedrock.client:  - inferenceConfig: {'temperature': 0, 'topP': 0.1}
DEBUG:idp_common.bedrock.client:  - system: [{'text': 'You are a document classification expert who can analyze and classify multiple documents and their page boundaries wit

Testing invokation #2, with CACHEPOINT


INFO:idp_common.bedrock.client:Bedrock request successful after 1 attempts. Duration: 17.70s
DEBUG:idp_common.bedrock.client:Response: {'ResponseMetadata': {'RequestId': '8d64f445-0ad6-4b29-9f01-98b4f84440e3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 22 May 2025 20:56:51 GMT', 'content-type': 'application/json', 'content-length': '2530', 'connection': 'keep-alive', 'x-amzn-requestid': '8d64f445-0ad6-4b29-9f01-98b4f84440e3'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': 'I\'ll analyze this document package to identify and classify the different document types within it.\n\nLooking through the document, I can identify several distinct document segments:\n\n1. Pages 1: A formal letter from the Western Dark Fired Tobacco Growers\' Association to Senator Wendell Ford\n2. Page 2: A lab services consistency report form\n3. Page 3: An email forwarded from Byron Nelson regarding "Morning Team Notes"\n4. Page 4: What appears to be a lab test r


Response summary:
Usage: {'inputTokens': 5100, 'outputTokens': 604, 'totalTokens': 6751, 'cacheReadInputTokens': 1047, 'cacheWriteInputTokens': 0}


In [24]:
# Test with Nova
model_id = "us.amazon.nova-pro-v1:0"

In [25]:
invoke_with_caching(model_id, 1)

DEBUG:idp_common.bedrock.client:Found <<CACHEPOINT>> tags in text content: 
The <document-types> XML tags contain a markdown ...
DEBUG:idp_common.bedrock.client:Split text into 2 parts at cachepoint tags
DEBUG:idp_common.bedrock.client:Text part 1: 667 words
DEBUG:idp_common.bedrock.client:Inserting cachePoint #1 after text part 1
DEBUG:idp_common.bedrock.client:Text part 2: 2474 words
INFO:idp_common.bedrock.client:Processed content with 1 cachepoint insertions
INFO:idp_common.bedrock.client:Applied cachePoint processing for supported model: us.amazon.nova-pro-v1:0
INFO:idp_common.bedrock.client:Bedrock request attempt 1/8:
DEBUG:idp_common.bedrock.client:  - model: us.amazon.nova-pro-v1:0
DEBUG:idp_common.bedrock.client:  - inferenceConfig: {'temperature': 0, 'topP': 0.1}
DEBUG:idp_common.bedrock.client:  - system: [{'text': 'You are a document classification expert who can analyze and classify multiple documents and their page boundaries within a document package from various domain

Testing invokation #1, with CACHEPOINT


INFO:idp_common.bedrock.client:Bedrock request successful after 1 attempts. Duration: 2.87s
DEBUG:idp_common.bedrock.client:Response: {'ResponseMetadata': {'RequestId': 'e51e1ac8-1161-4c95-8d22-91ae13c727ee', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 22 May 2025 20:56:54 GMT', 'content-type': 'application/json', 'content-length': '1307', 'connection': 'keep-alive', 'x-amzn-requestid': 'e51e1ac8-1161-4c95-8d22-91ae13c727ee'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': '```json\n{\n    "segments": [\n        {\n            "ordinal_start_page": 1,\n            "ordinal_end_page": 2,\n            "type": "letter"\n        },\n        {\n            "ordinal_start_page": 3,\n            "ordinal_end_page": 4,\n            "type": "email"\n        },\n        {\n            "ordinal_start_page": 5,\n            "ordinal_end_page": 5,\n            "type": "invoice"\n        },\n        {\n            "ordinal_start_page": 6,\n            


Response summary:
Usage: {'inputTokens': 4627, 'outputTokens': 263, 'totalTokens': 6045, 'cacheReadInputTokens': 0, 'cacheWriteInputTokens': 1155}


In [26]:
invoke_with_caching(model_id, 2)

DEBUG:idp_common.bedrock.client:Found <<CACHEPOINT>> tags in text content: 
The <document-types> XML tags contain a markdown ...
DEBUG:idp_common.bedrock.client:Split text into 2 parts at cachepoint tags
DEBUG:idp_common.bedrock.client:Text part 1: 667 words
DEBUG:idp_common.bedrock.client:Inserting cachePoint #1 after text part 1
DEBUG:idp_common.bedrock.client:Text part 2: 2474 words
INFO:idp_common.bedrock.client:Processed content with 1 cachepoint insertions
INFO:idp_common.bedrock.client:Applied cachePoint processing for supported model: us.amazon.nova-pro-v1:0
INFO:idp_common.bedrock.client:Bedrock request attempt 1/8:
DEBUG:idp_common.bedrock.client:  - model: us.amazon.nova-pro-v1:0
DEBUG:idp_common.bedrock.client:  - inferenceConfig: {'temperature': 0, 'topP': 0.1}
DEBUG:idp_common.bedrock.client:  - system: [{'text': 'You are a document classification expert who can analyze and classify multiple documents and their page boundaries within a document package from various domain

Testing invokation #2, with CACHEPOINT


INFO:idp_common.bedrock.client:Bedrock request successful after 1 attempts. Duration: 2.09s
DEBUG:idp_common.bedrock.client:Response: {'ResponseMetadata': {'RequestId': 'a1adc106-6c31-4637-81f5-b85ac4230af6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 22 May 2025 20:56:57 GMT', 'content-type': 'application/json', 'content-length': '1307', 'connection': 'keep-alive', 'x-amzn-requestid': 'a1adc106-6c31-4637-81f5-b85ac4230af6'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': '```json\n{\n    "segments": [\n        {\n            "ordinal_start_page": 1,\n            "ordinal_end_page": 2,\n            "type": "letter"\n        },\n        {\n            "ordinal_start_page": 3,\n            "ordinal_end_page": 4,\n            "type": "email"\n        },\n        {\n            "ordinal_start_page": 5,\n            "ordinal_end_page": 5,\n            "type": "invoice"\n        },\n        {\n            "ordinal_start_page": 6,\n            


Response summary:
Usage: {'inputTokens': 4627, 'outputTokens': 263, 'totalTokens': 6045, 'cacheReadInputTokens': 1155, 'cacheWriteInputTokens': 0}


In [27]:
# test with a model that doesn't support caching
model_id = "us.amazon.nova-premier-v1:0"
invoke_with_caching(model_id, 1)

INFO:idp_common.bedrock.client:Bedrock request attempt 1/8:
DEBUG:idp_common.bedrock.client:  - model: us.amazon.nova-premier-v1:0
DEBUG:idp_common.bedrock.client:  - inferenceConfig: {'temperature': 0, 'topP': 0.1}
DEBUG:idp_common.bedrock.client:  - system: [{'text': 'You are a document classification expert who can analyze and classify multiple documents and their page boundaries within a document package from various domains. Your task is to determine the document type based on its content and structure, using the provided document type definitions. Your output must be valid JSON according to the requested format.'}]
DEBUG:idp_common.bedrock.client:  - messages: [{'role': 'user', 'content': [{'text': '\nThe <document-types> XML tags contain a markdown table of known doc types for detection.\n<document-types>\n| type | description |\n| --- | --- |\n| letter | A formal written message that is typically sent from one person to another |\n| form | A document with blank spaces for filli

Testing invokation #1, with CACHEPOINT


INFO:idp_common.bedrock.client:Bedrock request successful after 1 attempts. Duration: 9.44s
DEBUG:idp_common.bedrock.client:Response: {'ResponseMetadata': {'RequestId': 'ca73c97f-5c2a-4d92-87ab-ba2d4a96951b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 22 May 2025 20:57:08 GMT', 'content-type': 'application/json', 'content-length': '2448', 'connection': 'keep-alive', 'x-amzn-requestid': 'ca73c97f-5c2a-4d92-87ab-ba2d4a96951b'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': '```json\n{\n  "segments": [\n    {\n      "ordinal_start_page": 1,\n      "ordinal_end_page": 1,\n      "type": "letter"\n    },\n    {\n      "ordinal_start_page": 2,\n      "ordinal_end_page": 2,\n      "type": "specification"\n    },\n    {\n      "ordinal_start_page": 3,\n      "ordinal_end_page": 3,\n      "type": "email"\n    },\n    {\n      "ordinal_start_page": 4,\n      "ordinal_end_page": 4,\n      "type": "scientific_publication"\n    },\n    {\n      "ordi


Response summary:
Usage: {'inputTokens': 6143, 'outputTokens': 643, 'totalTokens': 6786}
